# Get Raw Images

This notebook will demonstrate how to search for a specific image in order to get its `sequence_id`.

Using a `sequence_id` we will create an `ObservationInfo` object for discovering information about the observation before showing how to download the image metadata and FITS files.

In [1]:
import pandas as pd
from astropy.coordinates import SkyCoord

from panoptes.data.search import search_observations
from panoptes.data.observations import ObservationInfo

## Search for observations

Using a sky coordinate, search for all matching observations with a certain radius.

In [2]:
search_coords = SkyCoord.from_name('M42')
search_coords

<SkyCoord (ICRS): (ra, dec) in deg
    (83.82, -5.39)>

In [3]:
search_results = search_observations(
    coords=search_coords,
    radius=10,
    min_num_images=25,
    # unit_id='PAN019'
)

print(f'Found {len(search_results)} observations')

Found 105 observations


In [4]:
# Show a random example (run this cell again to see a different entry)
search_results.sample(1).T

,5534
camera_id,ee04d1
camera_lens_serial_number,HA0522610
camera_serial_number,22071246706
coordinates_airmass,1.667149
coordinates_mount_dec,-5.39111
coordinates_mount_ha,3.197076
coordinates_mount_ra,83.82208
field_name,M42
image_type,SCIENCE
imageh,3476


In [5]:
# Show the first and last five entries.
search_results

,camera_id,camera_lens_serial_number,camera_serial_number,coordinates_airmass,coordinates_mount_dec,coordinates_mount_ha,coordinates_mount_ra,field_name,image_type,imageh,imagew,num_images,sequence_id,software_version,time,total_exptime,unit_id,exptime
7222,14d3bd,HA0028608,12070048413,1.208084,-6.229778,22.497552,76.08150,Wasp 35,SCIENCE,3476,5208,28,PAN001_14d3bd_20180113T052325,POCSv0.6.0,2018-01-13 05:23:25+00:00,3360.0,PAN001,120.000000
7221,ee04d1,HA0522610,22071246706,1.202114,-6.229778,22.541475,76.08150,Wasp 35,SCIENCE,3476,5208,28,PAN001_ee04d1_20180113T052325,POCSv0.6.0,2018-01-13 05:23:25+00:00,3368.4,PAN001,120.300000
7213,ee04d1,HA0522610,22071246706,1.112569,-6.229778,0.257038,76.08150,Wasp 35,SCIENCE,3476,5208,60,PAN001_ee04d1_20180114T052427,POCSv0.6.0,2018-01-14 05:24:27+00:00,7218.0,PAN001,120.300000
7214,14d3bd,HA0028608,12070048413,1.397889,-6.229778,21.570712,76.08150,Wasp 35,SCIENCE,3476,5208,60,PAN001_14d3bd_20180114T052427,POCSv0.6.0,2018-01-14 05:24:27+00:00,7200.0,PAN001,120.000000
7193,14d3bd,HA0028608,12070048413,1.110339,-6.229778,23.965838,76.08150,Wasp 35,SCIENCE,3476,5208,55,PAN001_14d3bd_20180116T054110,POCSv0.6.0,2018-01-16 05:41:10+00:00,6600.0,PAN001,120.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1924,42433a,RL1548256,273074013260,1.340459,-5.391110,2.288399,83.82208,M42,NaN,0,0,64,PAN019_42433a_20220117T082323,POCSv0.8.0.post0.dev7+g8c57c125.dirty,2022-01-17 08:23:23+00:00,5760.0,PAN019,90.000000
1915,42433a,RL1548256,273074013260,1.347575,-5.391110,21.719481,83.82208,M42,NaN,0,0,62,PAN019_42433a_20220118T045650,POCSv0.8.0.post0.dev9+gce63321b,2022-01-18 04:56:50+00:00,5550.0,PAN019,89.516129
1914,c623e9,RL1552832,273074013304,1.186343,-5.391110,22.599409,83.82208,M42,NaN,0,0,62,PAN019_c623e9_20220118T045650,POCSv0.8.0.post0.dev9+gce63321b,2022-01-18 04:56:50+00:00,5518.0,PAN019,89.000000
7,608b40,RG0225767,592076084842,1.858007,-5.391110,20.765004,83.82208,M42,SCIENCE,4016,6020,37,PAN026_608b40_20231114T163852,POCSv0.8.0.dev126+gf7027455,2023-11-14 16:38:52+00:00,1110.0,PAN026,30.000000


## Select observation

A `sequence_id` is used to explore observations. Here we take the most recent observation to get the `sequence_id`.

In [6]:
# We choose an entry that only has a few images while running on binder and get the most recent.
record = search_results.iloc[-1]
record.T

camera_id                                           ecb3c3
camera_lens_serial_number                        RG0446628
camera_serial_number                          632076015094
coordinates_airmass                               1.727487
coordinates_mount_dec                             -5.39111
coordinates_mount_ha                             21.005468
coordinates_mount_ra                              83.82208
field_name                                             M42
image_type                                         SCIENCE
imageh                                                4016
imagew                                                6020
num_images                                              37
sequence_id                  PAN026_ecb3c3_20231114T163852
software_version               POCSv0.8.0.dev126+gf7027455
time                             2023-11-14 16:38:52+00:00
total_exptime                                       1110.0
unit_id                                             PAN0

## Get Observation info

We can get more information about the observation by creating an `ObservationInfo` object. You can create it with just a `sequence_id` (e.g. `ObservationInfo(sequence_id=record.sequence_id)) but here we pass all the meta data.

In [7]:
obs_info = ObservationInfo(meta=record)

# Show the observation object.
obs_info

Obs: seq_id=PAN026_ecb3c3_20231114T163852 num_images=37

In [8]:
obs_info.meta

camera_id                                           ecb3c3
camera_lens_serial_number                        RG0446628
camera_serial_number                          632076015094
coordinates_airmass                               1.727487
coordinates_mount_dec                             -5.39111
coordinates_mount_ha                             21.005468
coordinates_mount_ra                              83.82208
field_name                                             M42
image_type                                         SCIENCE
imageh                                                4016
imagew                                                6020
num_images                                              37
sequence_id                  PAN026_ecb3c3_20231114T163852
software_version               POCSv0.8.0.dev126+gf7027455
time                             2023-11-14 16:38:52+00:00
total_exptime                                       1110.0
unit_id                                             PAN0

### Show example of the image metadata and how to save to file

The metadata for each image is stored in the `obs_info.image_metadata` dataframe. 

We view a random example and then write the metadata to a file that can be downloaded in the file browser on the left.

In [9]:
# Randomly sample one image from the dataframe then transpose it so it's easier to read.
obs_info.image_metadata.sample(1).T

time,2023-11-14 16:52:33+00:00
file_creation_date,2023-11-14 16:52:33+00:00
uid,PAN026_ecb3c3_20231114T163852_20231114T165233
unit_id,PAN026
sequence_id,PAN026_ecb3c3_20231114T163852
image_type,SCIENCE
raw_url,https://storage.googleapis.com/panoptes-images...
status,SOLVED
MJD_OBS,51545.258958
environment_moonsep,41.249006
environment_moonfrac,0.020392


In [10]:
obs_info.image_metadata.to_csv(f'{obs_info.sequence_id}-image-metadata.csv')

### Download files locally

We can use the built-in `download_images` method of our `obs_info` object to download the files locally.

By default this will create an output directory that is the same as the `sequence_id` but this can be changed. You can also pass a limited list of files for download. 

Run `obs_info.download_images?` for more information.

In [11]:
# image_files = obs_info.download_images()

### Write image list to a CSV file, use `wget` to download

Or, if you prefer you can just get a list of files saved as csv and work on them in a command line terminal.

This first creates a DataFrame then immediately writes that to a CSV file. There are many ways to accomplish this but this is an easy way.

We then view the last few lines of the file using the command line tool `tail` called in a cell with the `!` operator.

We then use the same `!` operator to call `wget` in the shell and download a list of files. Note that you can just copy the CSV file to your computer or server and use the `wget -i <filename.txt>` command to download all the files. **Note that this can be a lot of data!**

In [12]:
# Create a dataframe and then write it to a CSV file.
pd.DataFrame(obs_info.get_image_list()).to_csv(f'{obs_info.sequence_id}-raw.txt', index=False, header=False)

In [13]:
# View the last entries in the file.
!tail {obs_info.sequence_id}-raw.txt

https://storage.googleapis.com/panoptes-images/PAN026/ecb3c3/20231114T163852/20231114T170237.fits.fz
https://storage.googleapis.com/panoptes-images/PAN026/ecb3c3/20231114T163852/20231114T170323.fits.fz
https://storage.googleapis.com/panoptes-images/PAN026/ecb3c3/20231114T163852/20231114T170408.fits.fz
https://storage.googleapis.com/panoptes-images/PAN026/ecb3c3/20231114T163852/20231114T170524.fits.fz
https://storage.googleapis.com/panoptes-images/PAN026/ecb3c3/20231114T163852/20231114T170609.fits.fz
https://storage.googleapis.com/panoptes-images/PAN026/ecb3c3/20231114T163852/20231114T170655.fits.fz
https://storage.googleapis.com/panoptes-images/PAN026/ecb3c3/20231114T163852/20231114T170741.fits.fz
https://storage.googleapis.com/panoptes-images/PAN026/ecb3c3/20231114T163852/20231114T170827.fits.fz
https://storage.googleapis.com/panoptes-images/PAN026/ecb3c3/20231114T163852/20231114T170940.fits.fz
https://storage.googleapis.com/panoptes-images/PAN026/ecb3c3/20231114T163852/20231114T17102

#### Download the data

In [14]:
### Uncomment to run ###
# !wget -i {obs_info.sequence_id}-raw.txt
